In [1]:
from database import *
from backtest import *

/Users/raychung/opt/anaconda3/lib/python3.8/site-packages/shioaji/__init__.py:20: Warning: Mac OS currently don't provide ca module all function about placing order is not working.
  warnings.warn("Mac OS currently don't provide ca module all function about placing order is not working.", Warning)


In [ ]:
api = sj.Shioaji()
api.login(
    person_id="YOUR_ID", 
    passwd="YOUR_PASSWORD", 
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)
# api.logout()

connection = sqlite3.connect('data.db')

In [16]:
ticks = {}
kbars = {}
volume_today = {}

In [17]:
def tbot(code):

    print('=== 啟動 tbot #{} ==='.format(code))
    
    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    if date not in tw_calendar.opens:
        print('今日非交易日')
        return 

    ticks[code] = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    ticks[code] = ticks[code][['close', 'volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()

In [18]:
code = '2330'
tbot(code)

=== 啟動 tbot #2330 ===


In [19]:
ticks
ticks[code]
kbars
kbars[code]
volume_today 

{'2330': 15248}

In [20]:
@api.quote.on_quote
def quote_callback(topic: str, quote: dict):
    
#     print('\n\ntopic:', topic)
#     print('quote:', quote)
    
    code = topic.split('/')[-1]
    
    ts = pd.to_datetime(quote.get('Date') + ' ' + quote.get('Time'))
    
    if ts.hour < 9:
        return 
    
    close = quote.get('Close')[0]
    volume = quote.get('Volume')[0]
    vol_sum = quote.get('VolSum')[0]
    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

In [21]:
api.quote.subscribe(api.Contracts.Stocks[code], quote_type='tick')

Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/2330 | Event: Subscribe or Unsubscribe ok


In [43]:
api.quote.unsubscribe(api.Contracts.Stocks[code], quote_type='tick')

Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/2330 | Event: Subscribe or Unsubscribe ok


In [42]:
# ticks[code]
volume_today[code]

22454

In [44]:
import time

def tbot(code):

    print('=== 啟動 tbot #{} ==='.format(code))
    
    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    if date not in tw_calendar.opens:
        print('今日非交易日')
        return 

    ticks[code] = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    ticks[code] = ticks[code][['close', 'volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    
    api.quote.subscribe(api.Contracts.Stocks[code], quote_type='tick')
    
    while True:
            
        time.sleep(1)
        
        current_time = datetime.datetime.now()
        
        if current_time.second == 0:

            kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
            kbars[code] = kbars[code][date:]
            
            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')
            
            try:
                current_kbar = kbars[code].loc[str_current_time]
            except:
                continue
            
            print(current_kbar)

In [ ]:
tbot('2330')